# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.environ["LANGCHAIN_API_KEY"]

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'search(query: str) -> str - Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic.v1 import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [12]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [13]:
get_current_temperature.name

'get_current_temperature'

In [14]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [15]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [16]:
from langchain.tools.render import format_tool_to_openai_function

In [17]:
format_tool_to_openai_function(get_current_temperature)

/Users/malcolm/.pyenv/versions/anaconda3-2023.03-1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [18]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 34.7°C'

In [19]:
! pip install wikipedia

In [20]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [21]:
search_wikipedia.name

'search_wikipedia'

In [22]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [23]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [24]:
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text

In [25]:
!pip install openapi-schema-pydantic openapi-pydantic

In [26]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain.utilities.openapi import OpenAPISpec
from langchain_community.utilities.openapi import OpenAPISpec
# from langchain_community.tools.openapi.utils.openapi_utils import OpenAPISpec
from openapi_pydantic import OpenAPI, Info, PathItem, Operation, Response
OpenAPISpec

langchain_community.utilities.openapi.OpenAPISpec

In [27]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [28]:
spec = OpenAPISpec.from_text(text)

In [29]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [30]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [31]:
from langchain.chat_models import ChatOpenAI

In [32]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

/Users/malcolm/.pyenv/versions/anaconda3-2023.03-1/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [33]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 123, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0ef231ed-d2e1-4adc-bb78-5221f24e491f-0')

In [34]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 126, 'total_tokens': 145}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-860f5bc7-9ac6-40bb-a9d9-9bc053ee367f-0')

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [35]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [36]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 129, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f7b68e97-37c2-4fd5-b813-e7ef4687c890-0')

In [37]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 125, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d932704c-cd31-43d5-820a-0fe0a74e5f11-0')

In [38]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [39]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 137, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-d501d3e1-418f-4288-a915-d873cccb4a7d-0')

In [40]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [41]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [42]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [43]:
# This is agent action because what its doing is calling one of the tools we created
type(result)

langchain_core.agents.AgentActionMessageLog

In [44]:
# Whos us exactly what tool its calling
result.tool

'get_current_temperature'

In [45]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [46]:
get_current_temperature(result.tool_input)

'The current temperature is 12.3°C'

In [47]:
result = chain.invoke({"input": "hi!"})

In [48]:
type(result)

langchain_core.agents.AgentFinish

In [49]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [50]:
from langchain.schema.agent import AgentFinish
def route(result):
    """Act on the result of the agent chain and look up correct tool to use and invoke."""
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [51]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [52]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [53]:
result

'The current temperature is 12.3°C'

In [54]:
result = chain.invoke({"input": "What is langchain?"})

In [55]:
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Prompt engineering\nSummary: Prompt engineering is the process of structuring text that can be interpreted and understood by a generative AI model. A prompt is natural language text

In [56]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'